In [ ]:
system"cd ",getenv[`HOME],"/course-introductory-workshop"
.trn.nbdir:system"cd"
\l scripts/loaddata.q

**Learning objectives**

To understand:
* How to call functions
* How to define user defined functions
* Creating and applying projections
* Iterations 

## Functions

So far we have used built-in functions. Now we introduce user-defined functions.

### Calling functions

Functions are called with the arguments in square brackets `[]`. For example we can call the builtin `max` function on a list like so:

In [ ]:
max[10 11 12]

With a unary (single argument) function, we can omit the square brackets. These two lines are equivalent:

In [ ]:
max[10 11 12]  // functional notation
max 10 11 12   // infix notation

### Defining functions

We can define our own [functions](https://code.kx.com/q/basics/function-notation/). 

Here is a binary (two-argument) function that calculates the speed in km per hour from distance traveled (miles) and duration (hours).

In [ ]:
speed:{[miles;hours]
 mph:miles%hours;
 kph:1.609*mph;
 // return the speed in kph
 :kph;
 }

Here we refer to `miles` and `hours` as arguments of the `speed` function. `kph` is a local variable we define inside the function. We then [explicitly return](https://code.kx.com/q/basics/function-notation/#explicit-return) it as the function’s result using `:`.

 <img src="images/qbies.png" width="50px" align="left"/><p style='color:#273a6e'><i> A function definition is a list of expressions, separated by semicolons and embraced by curly brackets. Functions can be defined over multiple lines: each line, except for the first one, must start with at least one whitespace character (we recommend two). This includes the line with the closing curly bracket. The arguments listed in the [signature](https://code.kx.com/q/basics/function-notation/#signature) are embraced in square brackets and separated by semicolons. </i></p>

An example call to the `speed` function looks like this:

In [ ]:
speed[15;0.5]

If there is no [explicit return](https://code.kx.com/q/basics/function-notation/#explicit-return) from a function its result is the result of evaluating the last expression in it. So the code above can be rewritten as 

In [ ]:
speed:{[miles;hours]
 1.609*miles%hours  // NOTE NO SEMICOLON
 }
speed[15;.5]        // result is unchanges

We can also call this function with a list of distances and a corresponding list of durations.

In [ ]:
speed[15 30 20;.5 .9 1.0]

###### Exercise 19
Create the following function: <br>
_func_ which is equivalent to the mathematical function: <br>\begin{equation}
res= - \frac{y(x+1)^2}{(2(x+1))-1}
\end{equation}
(What is the minimum number of brackets you have to use?) <br> 

In [ ]:
 //Use [`xexp`](https://code.kx.com/q/ref/exp/) for power and `%` for division;
{neg (y*(1+x) xexp 2) %-1+2*1+x}

In [ ]:
// Enter your code here 

In [ ]:
ex19[10;5] //check correct output

### Explicit and Implicit parameters

In the `speed` function above, we have named its arguments. Call these explicit arguments. 

In [ ]:
speed:{[miles;hours] //explicit parameters
 1.609*miles%hours 
 }

When functions have no more than three arguments, their names can be omitted, and `x`,`y`, and `z` used as implicit arguments. So our `speed` function can be written as:

In [ ]:
speed:{1.609*x%y}

Here we calculate the speed for two distances for a single duration.

In [ ]:
speed[15 30;.5] 

###### Exercise 20

Create a function that will find the area of a rectangle with length 7.93 and width 1.87 using implicit parameters.

In [ ]:
rectangleArea2:{x*y}  //using implicit parameters
rectangleArea2[7.93;1.87] 

In [ ]:
// Enter your code here 

In [ ]:
ex20[] //check correct output

### Call functions from qSQL

So far our arguments have been lists and atoms. We can also call functions in qSQL queries.


In [ ]:
jan09:select from trips where date within 2009.01.01 2009.01.07
select spd:speed[distance;duration % 0D01:00],distance,duration from jan09 where vendor = `VTS

This gives us the speed in km/h for each trip in our `jan09` table. We use `duration % 0D01:00` to give us a number of hours as a floating point number from the nanosecond precision duration we have stored in the trips table.

We can combine our functions, native kdb+/q functions, and qSQL with grouping and aggregation to get an average speed for each vendor in our `jan09` table.

In [ ]:
select avgspeed:speed[sum distance;sum[duration]%0D01:00] by vendor from jan09

###### Exercise 21

- Create a function called `tipOverDistance` that divides explicit argument `x` by argument `y`.

In [ ]:
tipOverDistance:{[x;y] x%y}  

In [ ]:
//Enter your code here

In [ ]:
ex21_a[3;2] //check correct output

- Write a function `createTable` that selects from `jan09` the columns `vendor`, `distance`, and `tip`; and adds a new column from the result of `tipOverDistance` applied to columns `tip` and `distance`.  

In [ ]:
createTable:{
  select tipPerDist:tipOverDistance[tip;distance], distance, tip, vendor from jan09 where distance > 0
 }

In [ ]:
//Enter your code here

In [ ]:
ex21_b[] //check correct output

- Find the average tip per mile per vendor from the result of `createTable`.

In [ ]:
select avg tipPerDist, avg distance, avg tip by vendor from createTable[]

In [ ]:
//Enter your code here

In [ ]:
ex21_c[] //check correct output

## Iterators

Most iteration is handled implicitly by q operators and keywords. Beyond that, we have iterators. An iterator is an operator that modifies how a function is applied.

Say we want to add `1 2` to `3 4 5`

In [ ]:
1 2+3 4 5

This signals a length error. The Add operator iterates implicitly, but expects its arguments [to be atoms or have matching lengths](https://code.kx.com/q/basics/conformable/). 

We clearly have something else in mind. Using iterators, we can modify the application of Add to add both vectors together. 

### Mapping iterators

In this instance, we can use:

+ [Each-right and Each-left](https://code.kx.com/q/ref/maps/#each-left-and-each-right)  

<img src="images/eachRighteachLeft.png" width="400" height="200">

In [ ]:
1 2+\: 3 4 5 //each left
1 2+/: 3 4 5 //each right

Each Right and Each Left are both examples of **map iterators**, the simplest kdb+/q iterators. Other map iterators are:
* Each
* Each Prior

Below we have a list of lists, we can use the keyword `count` to see how many items are in the list:

In [ ]:
L:("the";"quick";"brown";"fox")
count L

The `each` keyword modifies only unary (single argument) functions, as above. To modify a multivalent function, we use the Each operator. Let’s look at the Take operator `#` which gets a subset of the data:  

In [ ]:
count each L
type each L //checking the type all element

`each` can only modifies monadic functions (1 parameter) as seen above. To modify a multivalent function, we can use `each-both`. Let's look at the function `#` which gets a subset of the data:  

In [ ]:
3#L                               // returned the first three items of the list
3#'L                              // returned the first three items of each item

Can you predict what `3#''L` returns? Try it.

In [ ]:
// Enter your code here 

###### Exercise 23

Create two lists `x: 10 30 20 40, y: 13 34 25 46` and join them item by item, returning a pair of lists (type `0h`)

In [ ]:
x: 10 30 20 40
y: 13 34 25 46
x,'y
type x,'y

In [ ]:
//Enter your code here

In [ ]:
ex23[] //check correct output

### Accumulating iterators

Where map iterators apply a function *across argument items*, the accumulators apply it repeatedly to the results of successive evaluations. The function is first applied to the entire (first) argument; then to the result of that; then to the result of that; and so on. 

There are two accumulators in q. They both apply a function the same way; but one returns the result of each iteration; the other only the result of the last iteration. The iterators are: 
+ Scan (\\) 
+ Over (/)

<img src="images/scanIteration.png" width="500" height="200">

In [ ]:
N:1 4 7 10                         / numeric list
+/[N]                             / sum      (Over)   
+\[N]                             / sum      (Scan)   
*/[N]                             / product  (Over)
*\[N]                             / products (Scan)

What is the result of `-/[N]`? Try it.

In [ ]:
// Enter your Q code here 

Below are more examples with a different syntax format:

``function/[data]``

```(function/) data```

Both forms are valid; use which you prefer. 

In [ ]:
(+/)1 2 3 4 / Add these numbers, fold '+' over the vector; fold is sometimes called reduce or inject
(*/)1 2 3 4 / Extends to all functions in the expected way
sum 1 2 3 4 / Another way to sum the values, using a built-in function
(+\)1 2 3 4 / Cumulative sums, using scan
sums 1 2 3 4 / Same, using the built-in function

###### Exercise 24

a. Create a new function, add:{x+x}, and iterate this list of integers across it, 3 6 8

In [ ]:
//create a new function, add:{x+x}, and iterate this list of integers across it, 3 6 8
add:{x+x}
add each 3 6 8

In [ ]:
// Enter your code here 

In [ ]:
ex24_a[] //check correct output

b. Create a new function, add2:{x+y}, and iterate this list of integers across it, (3 6 8;4 7 9) so that the first value of each list is added together

In [ ]:
//create a new function, add2:{x+y}, and iterate this list of intergers across 
//it, (3 6 8;4 7 9) so that the first value of each list is added together
add2:{x+y}
'[add2][3 6 8;4 7 9]
add2'[3 6 8;4 7 9]

In [ ]:
// Enter your code here 

In [ ]:
ex24_b[] //check correct output

c. Multiply each value in this list, 3 5 4 2, against the value 11. Use both scan and over.

In [ ]:
 //multiply recursively by value in this list, 3 5 4 2, starting with a seed value of 11
//using scan 
{x*y}\[11;3 5 4 2]
//using over 
{x*y}/[11;3 5 4 2]

In [ ]:
// Enter your code here 

In [ ]:
ex24_c[] //check correct output

### Bonus Exercise -  Fibonacci sequence

Let's work towards generating the first 10 numbers of the Fibonacci sequence

>The Fibonacci sequence is defined such that each number is the sum of the two preceding ones, starting from 0 and 1. e.g. the first four numbers in the Fibonacci sequence is: 0 1 1 2

Tips:
1. Use sum and / (over) and # (take) to generate the Fibonacci numbers
2. you will also need to define and use a function
3. Start with 0 1
4. Use the [Do form of Over](https://code.kx.com/q/ref/accumulators/#do)

In [ ]:
exerFib:{{x, sum -2#x}/[x;0 1]}


x:0 1
sum -2#x
x,sum -2#x       / Beginning of Fibonacci sequence
{x,sum -2#x} 1 1 / Same, but as an (unnamed) function
fib:{x,sum -2#x} / Name it
fib/[10;1 1]     / Similar to +/ example earlier, apply the function repeatedly 10 times
10 fib/ 1 1      / Another way to invoke the function


In [ ]:
// Enter your code here 

In [ ]:
exerFib[20] //check correct output